# Supervised Learning to find value in stocks in the stock market

### Notebook by Catarina Fernandes, Diogo Almeida, Pedro Queirós
#### Supported by [Luís Paulo Reis](https://web.fe.up.pt/~lpreis/)
#### [Faculdade de Engenharia da Universidade do Porto](https://sigarra.up.pt/feup/pt/web_page.inicial)


**It is recommended to [view this notebook in nbviewer](http://nbviewer.ipython.org/github/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/example-data-science-notebook/Example%20Machine%20Learning%20Notebook.ipynb) for the best viewing experience.**

**You can also [execute the code in this notebook on Binder](https://mybinder.org/v2/gh/rhiever/Data-Analysis-and-Machine-Learning-Projects/master?filepath=example-data-science-notebook%2FExample%20Machine%20Learning%20Notebook.ipynb) - no local installation required.**

## Table of contents

1. [Introduction](#Introduction)

2. [Required libraries and Models](#Required-libraries-and-Models)
    - [Libraries](#Libraries)   
    - [Models](#Models)

3. [The problem domain](#The-problem-domain)

4. [Step 1: Data Analysis](#Step-1:-Data-Analysis)


## Introduction

[[ go back to the top ]](#Table-of-contents)



## Libraries

[[ go back to the top ]](#Table-of-contents)

If you don't have Python on your computer, you can use the [Anaconda Python distribution](http://continuum.io/downloads) to install most of the Python packages you need. Anaconda provides a simple double-click installer for your convenience.

This notebook uses several Python packages that come standard with the Anaconda Python distribution. The primary libraries that we'll be using are:

* **NumPy**: Provides a fast numerical array structure and helper functions.
* **pandas**: Provides a DataFrame structure to store data in memory and work with it easily and efficiently.
* **scikit-learn**: The essential Machine Learning package in Python.
* **matplotlib**: Basic plotting library in Python; most other Python plotting libraries are built on top of it.

To make sure you have all of the packages you need, install them with `conda`:

    conda install numpy pandas scikit-learn matplotlib seaborn
    
    conda install -c conda-forge watermark

`conda` may ask you to update some of them if you don't have the most recent version. Allow it to do so.

**Note:** I will not be providing support for people trying to run this notebook outside of the Anaconda Python distribution.

## The problem domain

[[ go back to the top ]](#Table-of-contents)


## Step 1: Answering the question

[[ go back to the top ]](#Table-of-contents)

The first step to any data analysis project is to define the question or problem we're looking to solve, and to define a measure (or set of measures) for our success at solving that task. The data analysis checklist has us answer a handful of questions to accomplish that, so let's work through those questions.

>Did you specify the type of data analytic question (e.g. exploration, association causality) before touching the data?

We're trying to design a predictive model capable of evaluating and finding stocks that are worth to invest on, based in the 10-K filings of each company.

>Did you define the metric for success before beginning?

Let's do that now. Since we're performing classification, we can use [accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision) — the fraction of correctly classified stocks — to quantify how well our model is performing. To ensure that the investers have profit, we should achieve an accuracy of 50% or greater.

>Did you understand the context for the question and the scientific or business application?

TODO

>Did you record the experimental design?

TODO

>Did you consider whether the question could be answered with the available data?

TODO

<hr />


## Step 1: Data Analysis

[[ go back to the top ]](#Table-of-contents)

Before start working with the data, we should analasy it and verify if there is erros, like null or undefined values.

In [12]:
import pandas as pd

ten_k_fillings_data = pd.read_csv('10_k_fillings.csv')
ten_k_fillings_data.head()


,Unnamed: 0,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,SG&A Expense,Operating Expenses,Operating Income,Earnings before Tax,Net Income,...,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Operating Cash Flow growth,Free Cash Flow growth,Receivables growth,Asset Growth,Book Value per Share Growth,SG&A Expenses Growth,class
0,INTC,7.084800e+10,0.1289,2.711100e+10,4.373700e+10,6.750000e+09,2.042100e+10,2.331600e+10,2.331700e+10,2.105300e+10,...,1.2513,-0.0191,-0.0277,0.3312,0.3793,0.1989,0.0382,0.1014,-0.0942,1
1,MU,3.039100e+10,0.4955,1.250000e+10,1.789100e+10,8.130000e+08,2.897000e+09,1.499400e+10,1.430300e+10,1.413500e+10,...,1.6100,0.0579,0.0650,1.1342,1.4922,0.4573,0.2275,0.6395,0.0942,1
2,AAPL,2.655950e+11,0.1586,1.637560e+11,1.018390e+11,1.670500e+10,3.094100e+10,7.089800e+10,7.290300e+10,5.953100e+10,...,0.2932,-0.0502,-0.0479,0.2057,0.2385,0.3734,-0.0256,-0.1584,0.0946,1
3,MSFT,1.103600e+11,0.1428,3.835300e+10,7.200700e+10,2.222300e+10,3.694900e+10,3.505800e+10,3.647400e+10,1.657100e+10,...,-0.3446,-0.0059,-0.0049,0.1108,0.0279,0.1806,0.0341,-0.0512,0.1144,1
4,HPQ,5.847200e+10,0.1233,4.780300e+10,1.066900e+10,4.859000e+09,6.605000e+09,4.064000e+09,3.013000e+09,5.327000e+09,...,1.2027,-0.0432,-0.0400,0.2314,0.2422,0.1584,0.0519,0.8039,0.1104,1


In [11]:
ten_k_fillings_data.describe()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,SG&A Expense,Operating Expenses,Operating Income,Earnings before Tax,Net Income,Net Income Com,...,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Operating Cash Flow growth,Free Cash Flow growth,Receivables growth,Asset Growth,Book Value per Share Growth,SG&A Expenses Growth,class
count,6.380000e+02,638.000000,6.380000e+02,6.380000e+02,6.380000e+02,6.380000e+02,6.380000e+02,6.380000e+02,6.380000e+02,6.380000e+02,...,638.000000,638.000000,638.000000,638.000000,638.000000,638.000000,638.000000,638.000000,638.000000,638.000000
mean,4.313656e+09,0.200593,2.418867e+09,1.893674e+09,6.838895e+08,1.209286e+09,6.849852e+08,6.876940e+08,5.387035e+08,5.365020e+08,...,0.367957,0.327554,0.354594,0.024223,0.342144,0.282030,0.160931,0.134593,0.112316,0.706897
std,1.832010e+10,1.236156,1.060182e+10,8.412074e+09,2.367724e+09,5.087444e+09,4.005216e+09,4.214649e+09,3.416454e+09,3.416642e+09,...,9.258839,4.700736,4.719772,10.419881,6.941349,1.567491,0.453783,1.487802,0.231320,0.455543
min,0.000000e+00,-0.959300,0.000000e+00,-2.970810e+08,7.926000e+05,-6.190000e+05,-1.268450e+09,-1.326000e+09,-4.964000e+09,-4.964000e+09,...,-136.000000,-0.158700,-0.229800,-197.027800,-31.936200,-1.000000,-0.864500,-17.090900,-0.641900,0.000000
25%,1.027300e+08,0.008600,4.501450e+07,3.723250e+07,2.861450e+07,4.583850e+07,-9.972250e+06,-1.322625e+07,-1.706438e+07,-1.767875e+07,...,-0.259475,-0.000375,-0.002425,-0.217250,-0.316125,-0.037050,-0.032350,-0.106900,-0.013300,0.000000
50%,4.539775e+08,0.100800,1.950135e+08,2.097740e+08,1.291985e+08,1.934000e+08,8.819500e+06,6.248500e+06,4.420444e+06,4.163274e+06,...,0.264250,0.014350,0.016250,0.118200,0.126250,0.099400,0.054200,0.035150,0.079900,1.000000
75%,1.896714e+09,0.231325,9.058808e+08,8.405540e+08,4.252228e+08,6.439558e+08,1.660810e+08,1.378770e+08,1.100322e+08,1.090343e+08,...,0.791325,0.060050,0.078225,0.497250,0.589150,0.282983,0.211925,0.174850,0.188800,1.000000
max,2.655950e+11,29.980400,1.637560e+11,1.018390e+11,2.445900e+10,8.131000e+10,7.089800e+10,7.290300e+10,5.953100e+10,5.953100e+10,...,100.000000,117.904200,117.904200,73.783500,148.819100,26.980800,3.653900,22.552600,1.944900,1.000000


In [13]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
sb.pairplot(ten_k_fillings_data)
;